In [72]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [73]:
pd.options.display.max_columns=None

In [74]:
df_esios=pd.read_csv('../data/datos_esios.csv', sep=';')

In [75]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42


In [76]:
df_esios.describe()

,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2
count,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000,26328.000000
mean,5476.657615,28672.335722,4413.087348,4421.192005,4189.052822,48.745913,42.104412,48.474281,48.483520
std,3185.745755,4615.591310,2217.773186,474.661673,469.408867,14.950758,20.792168,15.091625,15.106957
min,240.800000,17728.400000,546.000000,2461.500000,2315.100000,2.060000,-31.820000,0.000000,0.000000
25%,3015.600000,24733.475000,2318.000000,4125.800000,3797.900000,40.140000,29.747500,40.050000,40.100000
50%,4820.250000,28921.700000,4636.800000,4483.200000,4148.100000,49.565000,39.170000,49.000000,48.970000
75%,7285.800000,32265.825000,6286.600000,4799.700000,4705.300000,59.450000,51.560000,58.900000,58.742500
max,16952.600000,41297.900000,8768.300000,9481.400000,8270.600000,101.990000,874.010000,102.490000,103.610000


In [77]:
df_esios['datetime'].describe()  #tenemos repetidos los datos para el dia de 25 horas.

count                   26328
unique                  26325
top       2017-10-29 02:00:00
freq                        2
Name: datetime, dtype: object

In [79]:
df_esios['datetime']=pd.to_datetime(df_esios['datetime'])
df_esios.dtypes

datetime              datetime64[ns]
P48_EOLICA                   float64
P48_DEMANDA                  float64
P48_CARBON                   float64
POT_DISP_HULLA_ANT           float64
POT_DISP_HULA_SB             float64
PRECIO_MD_ESP                float64
PRECIO_MD_FR                 float64
PRECIO_I1                    float64
PRECIO_I2                    float64
dtype: object

#### 1) Columna a predecir:

"reservar"

Nuestra clasificación será:

* Si PMI1 > PMD -> 0

* Si PMI1 < PMD -> 1

* Si PMI1 == PMD:

    * Si PMI2 > PMD -> 0
    * Si PMI2 <= PMD -> 1

En la descarga y exploración de datos esios se obtuvo que de los 26.328 datos, hay 3.498 casos en los que el precio del mercado diario coincidió con el del intra, y de esos, 794 horas en las que además el intradiario 2 tampoco subió/bajó con respecto al diario.

In [80]:
df_esios_precio=df_esios[['PRECIO_MD_ESP','PRECIO_I1','PRECIO_I2','PRECIO_MD_FR']]

In [81]:
def reserva(dataframe_precios):
    reserva=np.zeros(len(dataframe_precios['PRECIO_MD_ESP']))
    for k,v in dataframe_precios.iterrows():
        a=v['PRECIO_MD_ESP']
        b=v['PRECIO_I1']
        c=v['PRECIO_I2']
        if (a<b) | ((a==b) & (c>b)):
            reserva[k]=0
        else:
            reserva[k]=1  
    df_esios['RESERVA']=reserva

In [82]:
reserva(df_esios_precio)

In [83]:
df_esios.sample(10)  #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA
12143,2017-02-16 22:00:00,2490.1,32279.8,7247.4,4799.7,4012.8,62.15,55.59,62.16,62.15,0.0
23005,2018-05-15 13:00:00,6705.7,31521.8,2187.3,4487.7,3796.7,53.87,41.71,53.37,54.86,1.0
24024,2018-06-27 00:00:00,3200.1,28499.2,3970.4,4799.7,3797.9,62.32,43.17,61.42,59.78,1.0
21936,2018-04-01 00:00:00,5532.0,25128.6,1175.8,4444.6,4705.3,40.43,40.43,39.43,44.77,1.0
20508,2018-01-31 11:00:00,2363.0,36101.8,5301.0,3329.5,4705.3,58.67,35.76,59.82,59.99,0.0
7110,2016-07-22 06:00:00,4770.9,26974.6,4543.6,4829.4,4135.3,34.50,32.36,35.69,34.50,0.0
18625,2017-11-14 00:00:00,11357.5,26350.8,5977.7,4463.6,4355.1,48.19,53.94,48.19,51.79,0.0
2627,2016-01-17 10:00:00,4001.7,28443.9,2212.1,4686.0,4147.8,37.06,37.06,38.75,39.67,0.0
10394,2016-12-06 01:00:00,2105.3,24139.0,6820.8,4799.7,3238.4,60.23,50.00,54.23,56.87,1.0
26076,2018-09-20 12:00:00,1339.5,35116.0,6670.3,4094.0,4705.3,74.73,64.94,76.18,75.71,0.0


#### 2) Interconexión

Creamos columna ACOPLADO_FR:

* Si Precio FR MD == Precio España MD están acoplados (1) si no no (0)*

In [84]:
def interconexion(dataframe_precios_inter):
    acoplado=np.zeros(len(dataframe_precios_inter['PRECIO_MD_ESP']))
    for k,v in dataframe_precios_inter.iterrows():
        a=v['PRECIO_MD_ESP']
        b=v['PRECIO_MD_FR']
        if (a==b):
            acoplado[k]=1
        else:
            acoplado[k]=0  
    df_esios['ACOPLADO_FR']=acoplado

In [85]:
interconexion(df_esios_precio)

In [86]:
df_esios.sample(10) #comprobamos aleatoriamente que ok

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR
5898,2016-06-01 18:00:00,5709.5,29582.0,1893.9,3951.9,4135.3,39.01,32.66,37.91,37.47,1.0,0.0
1869,2015-12-16 20:00:00,4822.3,35380.9,8356.4,5027.1,4705.3,65.98,39.27,63.98,63.48,1.0,0.0
580,2015-10-24 04:00:00,4014.4,20786.4,5006.4,5064.0,4705.3,39.58,30.34,39.48,39.50,1.0,0.0
12369,2017-02-26 08:00:00,2069.5,22389.1,3578.0,4829.4,4147.8,42.19,42.19,42.19,42.19,1.0,1.0
18972,2017-11-28 11:00:00,3146.8,36080.9,7787.9,4449.1,4143.3,76.34,76.34,78.43,79.34,0.0,1.0
21387,2018-03-09 02:00:00,9014.4,24356.2,1531.0,4799.7,4705.3,37.21,37.21,36.21,36.21,1.0,1.0
7391,2016-08-02 23:00:00,4415.9,30508.6,5908.0,4126.9,3577.8,40.69,28.98,40.72,41.03,0.0,0.0
17000,2017-09-07 08:00:00,4880.3,30580.9,4081.4,4529.7,4705.3,51.00,41.62,48.70,47.68,1.0,0.0
14901,2017-06-11 21:00:00,7083.8,28675.7,3937.9,3850.5,3795.6,44.09,33.40,44.09,45.67,0.0,0.0
25940,2018-09-14 20:00:00,4934.2,33288.7,6378.6,4442.3,4353.1,74.74,74.74,73.99,73.99,1.0,1.0


#### 3) MW Carbón disponible

Sumamos la potencia disponible de carbón. Nos interesa el total de MW disponibles.

In [87]:
df_esios['POT_DISP_CARBON']=[x['POT_DISP_HULLA_ANT'] + x['POT_DISP_HULA_SB'] for _,x in df_esios.iterrows()]

In [88]:
df_esios.head()

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7


#### 4) Datos pasados

Necesitamos el carbón casado el día anterior (ya que el carbón que se va a casar ese día no lo conocemos) así que tendremos que restar 1 día a la fecha para la columna P48_CARBON, creando la columna adicional P48_CARBON_D-1

Aprovechamos esta operación para crear la columna RESULTADO D-1 que es la columna reserva del día anterior.

In [97]:
df_esios_ant= df_esios[['datetime','P48_CARBON','RESERVA']].copy()

In [98]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-09-30 00:00:00,7181.4,0.0
1,2015-09-30 01:00:00,6771.7,0.0
2,2015-09-30 02:00:00,6559.5,0.0
3,2015-09-30 03:00:00,6560.4,0.0
4,2015-09-30 04:00:00,6724.1,0.0


In [99]:
df_esios_ant.dtypes

datetime      datetime64[ns]
P48_CARBON           float64
RESERVA              float64
dtype: object

In [100]:
dia=timedelta(days=1) 

fecha=pd.to_datetime(df_esios_ant['datetime'])
fecha_ant=pd.Series([x + dia for x in fecha])
df_esios_ant['datetime']=fecha_ant
df_esios_ant.tail()

,datetime,P48_CARBON,RESERVA
26323,2018-10-01 19:00:00,4861.1,0.0
26324,2018-10-01 20:00:00,4782.9,0.0
26325,2018-10-01 21:00:00,4826.0,0.0
26326,2018-10-01 22:00:00,4169.8,1.0
26327,2018-10-01 23:00:00,3775.5,1.0


In [101]:
df_esios_ant.head()

,datetime,P48_CARBON,RESERVA
0,2015-10-01 00:00:00,7181.4,0.0
1,2015-10-01 01:00:00,6771.7,0.0
2,2015-10-01 02:00:00,6559.5,0.0
3,2015-10-01 03:00:00,6560.4,0.0
4,2015-10-01 04:00:00,6724.1,0.0


In [102]:
df_esios_ant.columns=['datetime','P48_CARBON_D-1','RESERVA_D-1']

In [103]:
df_esios_ant.head()

,datetime,P48_CARBON_D-1,RESERVA_D-1
0,2015-10-01 00:00:00,7181.4,0.0
1,2015-10-01 01:00:00,6771.7,0.0
2,2015-10-01 02:00:00,6559.5,0.0
3,2015-10-01 03:00:00,6560.4,0.0
4,2015-10-01 04:00:00,6724.1,0.0


In [104]:
df_esios2=df_esios.merge(df_esios_ant,how='left',on='datetime')

In [107]:
df_esios2.head(48)

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN
5,2015-09-30 05:00:00,2777.9,22370.5,7174.6,5355.4,4705.3,45.65,30.55,50.65,50.65,0.0,0.0,10060.7,NaN,NaN
6,2015-09-30 06:00:00,2522.1,24532.6,7955.3,5355.4,4705.3,52.00,42.93,56.13,54.31,0.0,0.0,10060.7,NaN,NaN
7,2015-09-30 07:00:00,2303.5,27986.9,8281.3,5355.4,4705.3,59.90,53.78,62.00,62.40,0.0,0.0,10060.7,NaN,NaN
8,2015-09-30 08:00:00,2141.9,29690.2,8304.3,5355.4,4705.3,60.01,55.50,61.71,62.51,0.0,0.0,10060.7,NaN,NaN
9,2015-09-30 09:00:00,1980.8,30192.1,8263.3,5355.4,4705.3,60.60,49.27,62.86,63.03,0.0,0.0,10060.7,NaN,NaN


In [114]:
#comprobamos NaN por días 25-23 horas descuadrados

df_esios2[pd.isna(df_esios2['P48_CARBON_D-1'])]

,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON,P48_CARBON_D-1,RESERVA_D-1
0,2015-09-30 00:00:00,4727.7,24595.3,7181.4,5355.4,4705.3,48.20,38.51,54.13,56.80,0.0,0.0,10060.7,NaN,NaN
1,2015-09-30 01:00:00,4389.9,23083.0,6771.7,5355.4,4705.3,45.65,29.76,50.70,56.13,0.0,0.0,10060.7,NaN,NaN
2,2015-09-30 02:00:00,3889.6,22504.4,6559.5,5355.4,4705.3,45.35,28.01,50.35,53.96,0.0,0.0,10060.7,NaN,NaN
3,2015-09-30 03:00:00,3498.3,22297.5,6560.4,5355.4,4705.3,45.29,27.38,50.29,51.01,0.0,0.0,10060.7,NaN,NaN
4,2015-09-30 04:00:00,3032.9,22038.9,6724.1,5355.4,4705.3,45.42,27.00,49.88,50.42,0.0,0.0,10060.7,NaN,NaN
5,2015-09-30 05:00:00,2777.9,22370.5,7174.6,5355.4,4705.3,45.65,30.55,50.65,50.65,0.0,0.0,10060.7,NaN,NaN
6,2015-09-30 06:00:00,2522.1,24532.6,7955.3,5355.4,4705.3,52.00,42.93,56.13,54.31,0.0,0.0,10060.7,NaN,NaN
7,2015-09-30 07:00:00,2303.5,27986.9,8281.3,5355.4,4705.3,59.90,53.78,62.00,62.40,0.0,0.0,10060.7,NaN,NaN
8,2015-09-30 08:00:00,2141.9,29690.2,8304.3,5355.4,4705.3,60.01,55.50,61.71,62.51,0.0,0.0,10060.7,NaN,NaN
9,2015-09-30 09:00:00,1980.8,30192.1,8263.3,5355.4,4705.3,60.60,49.27,62.86,63.03,0.0,0.0,10060.7,NaN,NaN


In [118]:
df_esios[df_esios2['datetime'].duplicated()==True]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,datetime,P48_EOLICA,P48_DEMANDA,P48_CARBON,POT_DISP_HULLA_ANT,POT_DISP_HULA_SB,PRECIO_MD_ESP,PRECIO_MD_FR,PRECIO_I1,PRECIO_I2,RESERVA,ACOPLADO_FR,POT_DISP_CARBON
603,2015-10-25 02:00:00,4941.3,19699.0,4691.2,9481.4,8270.6,38.00,25.02,35.00,35.00,1.0,0.0,17752.0
628,2015-10-26 03:00:00,4996.9,19670.2,4070.0,5064.0,4135.3,32.99,31.09,25.10,32.99,1.0,0.0,9199.3
9508,2016-10-30 03:00:00,1695.5,19494.3,6054.4,3300.0,3573.3,46.90,31.40,46.04,46.90,1.0,0.0,6873.3
9533,2016-10-31 04:00:00,1797.7,19251.3,5183.9,3945.9,3341.9,45.43,34.39,45.29,45.43,1.0,0.0,7287.8
18245,2017-10-29 04:00:00,7148.1,19114.7,2102.9,3071.7,3227.9,39.93,23.19,36.86,30.00,1.0,0.0,6299.6
18270,2017-10-30 05:00:00,7411.5,20664.2,2640.6,4116.2,3785.1,33.90,33.90,31.50,30.50,1.0,1.0,7901.3
